В Colab код не заработает, так как использует десктопную графику! Можно использовать Anaconda на локальном компьютере.

Файл -> Скачать IPYNB

In [ ]:
!pip install tk

In [ ]:
from tkinter import *
import random
import numpy as np

width = 800
height = 600
root = Tk()

canvas = Canvas(root, width=width, height=height, bg='white')
canvas.pack()

tickrate = 100
tick_interval_ms = int(1000/tickrate)
radius = 10

coordinates = None

color_strings = ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]


#####################################

params = [
    {'var_name': 'G_attractive', 'start_value': 0, 'min': 0, 'max': 1.5},
    {'var_name': 'G_repulsive', 'start_value': 0.5, 'min': 0, 'max': 1.5},
    {'var_name': 'Pow_repulsive', 'start_value': 0.5, 'min': 0.1, 'max': 2},
    {'var_name': 'G_y', 'start_value': 0.2, 'min': -1, 'max': 2},
    {'var_name': 'v_multiplier', 'start_value': 0.95, 'min': 0, 'max': 1},
]
    
def register_global_var_with_slider(param):
    globals().__setitem__(param['var_name'], float(param['start_value']))
    slider = Scale(root, from_=param['min'], to=param['max'], orient=HORIZONTAL, resolution = 0.01,
               label=param['var_name'], command=lambda x: globals().__setitem__(param['var_name'], float(x)))
    slider.set(param['start_value'])
    slider.pack(side = LEFT)

for param in params:
    register_global_var_with_slider(param)

def update_coordinates(array, x_min, y_min, x_max, y_max):
    global color_strings, G_attractive, G_repulsive, Pow_repulsive, G_y, v_multiplier
    if array is None:
        count = 200
        x = np.random.uniform(low=x_min+radius, high=x_max-radius, size=count)
        y = np.random.uniform(low=y_min+radius-500, high=y_max-radius, size=count)
        v_x = np.random.uniform(low=-1, high=1, size=count)
        v_y = np.random.uniform(low=-1, high=1, size=count)
        colors = np.random.uniform(low=0, high=len(color_strings), size=count)
        distances_x = np.stack([x]*count, axis=1) - np.stack([x]*count, axis=0)
        distances_y = np.stack([y]*count, axis=1) - np.stack([y]*count, axis=0)
        distances_euclidean = np.maximum(0.1, np.sqrt(distances_x**2 + distances_y**2))
        np.fill_diagonal(distances_euclidean, np.inf)
        overlap_first, overlap_second = np.where(distances_euclidean < 2*radius)
        indices_to_delete = overlap_second[np.where(overlap_second > overlap_first)]
        x = np.delete(x, indices_to_delete)
        y = np.delete(y, indices_to_delete)
        v_x = np.delete(v_x, indices_to_delete)
        v_y = np.delete(v_y, indices_to_delete)
        colors = np.delete(colors, indices_to_delete)
    else:
        count = len(array)
        x = array[:, 0]
        y = array[:, 1]
        v_x = array[:, 2]
        v_y = array[:, 3]
        colors = array[:, 4]
        x += v_x
        y += v_y
        v_y += G_y
        v_y -= G_repulsive*np.maximum(0, y - y_max + radius)
        v_x -= G_repulsive*np.maximum(0, x - x_max + radius)
        v_x += G_repulsive*np.maximum(0, x_min + radius - x)
        distances_x = np.stack([x]*count, axis=1) - np.stack([x]*count, axis=0)
        distances_y = np.stack([y]*count, axis=1) - np.stack([y]*count, axis=0)
        distances_euclidean = np.maximum(0.1, np.sqrt(distances_x**2 + distances_y**2))
        np.fill_diagonal(distances_euclidean, np.inf)
        attractive_forces = G_attractive*300/distances_euclidean**2
        overlap = np.maximum(0, 2*radius - distances_euclidean)
        repulsive_forces = overlap ** Pow_repulsive
        forces = attractive_forces - repulsive_forces
        ratios = forces/distances_euclidean
        forces_x = distances_x*ratios
        forces_y = distances_y*ratios
        result_forces_x = np.sum(forces_x, axis=0)
        result_forces_y = np.sum(forces_y, axis=0)
        v_x += result_forces_x
        v_y += result_forces_y
        v_x *= v_multiplier
        v_y *= v_multiplier
    return np.stack([x, y, v_x, v_y, colors], axis=1)

#####################################

def tick():
    global coordinates, color_strings
    coordinates = update_coordinates(coordinates, 0, 0, width, height)
    canvas.delete(ALL)
    for x, y, v_x, v_y, color in coordinates:
        canvas.create_oval(x-radius, y-radius, x+radius, y+radius, width=1)
        canvas.create_oval(x-4, y-4, x+4, y+4, fill=color_strings[int(color)])

    canvas.after(tick_interval_ms, tick)

canvas.after(1, tick) 
root.mainloop()